<a href="https://colab.research.google.com/github/SidRaza786/Coding_ninjas_Demo/blob/master/Copy_of_Facial_Emotions_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 42.80 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [6]:
"/content/facial_expressions"

'/content/facial_expressions'

In [7]:
import csv
data = {}
with open("/content/facial_expressions/data/legend.csv") as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [8]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [9]:
import os

In [10]:
# make directory for store data
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [11]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training', emotion))
  os.mkdir(os.path.join('master_data/testing', emotion))

In [12]:
from shutil import copyfile

In [13]:
split_size = 0.8
for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('master_data/training', emotion, image)
    copyfile(source, dest)

  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('master_data/testing', emotion, image)
    copyfile(source, dest)

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [15]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation = 'relu', input_shape = (100, 100, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(8, activation = 'softmax'))
model.compile(optimizer= Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [16]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'
train_data_gen = ImageDataGenerator(rescale = 1.0/225)
train_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size = (100, 100),
                                                     class_mode = 'categorical',
                                                     batch_size = 128
                                                     )

test_data_gen = ImageDataGenerator(rescale = 1.0/225)
test_generator = test_data_gen.flow_from_directory(test_dir,
                                                     target_size = (100, 100),
                                                     class_mode = 'categorical',
                                                     batch_size = 128
                                                   )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [17]:
es = EarlyStopping(monitor = 'val_acc', patience=2, min_delta = 0.01)

In [18]:
model.fit_generator(train_generator, epochs = 10, verbose = 1, validation_data= test_generator, callbacks = [es])

<ipython-input-18-6cea27a4e0c6>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs = 10, verbose = 1, validation_data= test_generator, callbacks = [es])


Epoch 1/10
86/86 [==============================] - 24s 181ms/step - loss: 1.0032 - acc: 0.5468 - val_loss: 1.2479 - val_acc: 0.6200
Epoch 2/10
86/86 [==============================] - 15s 174ms/step - loss: 0.6291 - acc: 0.7809 - val_loss: 1.9961 - val_acc: 0.6360
Epoch 3/10
86/86 [==============================] - 15s 172ms/step - loss: 0.5394 - acc: 0.8155 - val_loss: 1.7110 - val_acc: 0.6747
Epoch 4/10
86/86 [==============================] - 15s 175ms/step - loss: 0.4681 - acc: 0.8394 - val_loss: 1.8623 - val_acc: 0.6740
Epoch 5/10
86/86 [==============================] - 15s 175ms/step - loss: 0.4331 - acc: 0.8483 - val_loss: 1.7967 - val_acc: 0.6721
